In [1]:
import os
from typing import overload
import numpy as np
from pymbar import MBAR
import pandas as pd

In [2]:
os.chdir('complex_6_20_200_200_restraint_loop_no_DI-PHE_0.01/loop_restraint_off_04_sir_this_way/post_processing')

In [3]:
#beta = 1.0 / (300 * 1.380649 * 6.02214076 / 1000)
beta = 1.0 / (300 * 1.380649e-23 )
name_list = [0.0, 0.2, 0.4, 0.6, 0.8, 1.0]
#name_list = [ 0.0, 0.01, 0.025, 0.05, 0.075, 0.1, 0.15, 0.2, 0.35, 0.5, 0.75, 1.0]
#name_list = [ 0.01, 0.025, 0.05, 0.075, 0.1, 0.15, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.85, 1]

# number of states
K = len(name_list)

mian_dir = os.getcwd()

In [4]:
os.chdir(mian_dir + '/' + str(name_list[0]))
result_path = os.getcwd() + '/MBAR_energy.txt'

In [5]:
u_kn = np.loadtxt(result_path)

In [6]:
u_kn.shape

(6000,)

In [7]:
for file in range(1,len(name_list)):

    os.chdir(mian_dir + '/' + str(name_list[file]))
    result_path = os.getcwd() + '/MBAR_energy.txt'

    a = np.loadtxt(result_path)
    #print(a.shape)
    #print(a)
    u_kn = np.vstack((u_kn,a))

In [8]:
u_kn.shape

(6, 6000)

In [9]:
N_k = np.full(K,u_kn.shape[1]/K, dtype=np.int32)
N_k

array([1000, 1000, 1000, 1000, 1000, 1000], dtype=int32)

In [10]:
mbar = MBAR(u_kn,N_k)

In [11]:
Delta_f_ij, dDelta_f_ij, Theta_ij = mbar.getFreeEnergyDifferences(return_theta=True)

# kbT  -->   kcal/mol   (  *  0.596  )

In [12]:
dDelta_f_ij

array([[0.        , 0.01030929, 0.01670425, 0.02083228, 0.02366974,
        0.02578144],
       [0.01030929, 0.        , 0.0069396 , 0.01173433, 0.01519446,
        0.01784393],
       [0.01670425, 0.0069396 , 0.        , 0.00506078, 0.0088807 ,
        0.0119006 ],
       [0.02083228, 0.01173433, 0.00506078, 0.        , 0.00396242,
        0.00718744],
       [0.02366974, 0.01519446, 0.0088807 , 0.00396242, 0.        ,
        0.00330578],
       [0.02578144, 0.01784393, 0.0119006 , 0.00718744, 0.00330578,
        0.        ]])

In [13]:
print('Delta_f_ij.shape:',Delta_f_ij.shape)
print(f"Restraint energy : {Delta_f_ij[0,-1] / beta / 1000 / 4.186 * 6.02214076e23:.3f} ± {dDelta_f_ij[0,-1] / beta / 1000 / 4.186 * 6.02214076e23:.3f} Kcal/mol")
with open(f'{mian_dir}/Restraint_energy.txt','w') as f:
    f.write(f'Restraint energy(MBAR): {Delta_f_ij[0,-1] / beta / 1000 / 4.186 * 6.02214076e23:.3f} ± {dDelta_f_ij[0,-1] / beta / 1000 / 4.186 * 6.02214076e23:.3f} Kcal/mol\n')
    for i in range(K-1):
        f.writelines(f'{Delta_f_ij[i][i+1] * 0.596:.3f}\n')

Delta_f_ij.shape: (6, 6)
Restraint energy : 1.874 ± 0.015 Kcal/mol


In [14]:
pd.DataFrame(Delta_f_ij * 0.596)

,0,1,2,3,4,5
0,0.000000,0.595961,1.040147,1.380074,1.650636,1.874384
1,-0.595961,0.000000,0.444186,0.784113,1.054675,1.278423
2,-1.040147,-0.444186,0.000000,0.339927,0.610488,0.834236
3,-1.380074,-0.784113,-0.339927,0.000000,0.270562,0.494310
4,-1.650636,-1.054675,-0.610488,-0.270562,0.000000,0.223748
5,-1.874384,-1.278423,-0.834236,-0.494310,-0.223748,0.000000


In [15]:
O_ij = mbar.computeOverlap()['matrix']
np.savetxt('../restraint_overlap.txt',O_ij,fmt='%.8f')
O_ij.round(decimals=3, out=O_ij)
pd.DataFrame(O_ij)

,0,1,2,3,4,5
0,0.341,0.233,0.159,0.113,0.085,0.068
1,0.233,0.205,0.174,0.147,0.128,0.113
2,0.159,0.174,0.176,0.171,0.164,0.157
3,0.113,0.147,0.171,0.184,0.191,0.194
4,0.085,0.128,0.164,0.191,0.210,0.223
5,0.068,0.113,0.157,0.194,0.223,0.245


In [16]:
dDelta_f_ij

array([[0.        , 0.01030929, 0.01670425, 0.02083228, 0.02366974,
        0.02578144],
       [0.01030929, 0.        , 0.0069396 , 0.01173433, 0.01519446,
        0.01784393],
       [0.01670425, 0.0069396 , 0.        , 0.00506078, 0.0088807 ,
        0.0119006 ],
       [0.02083228, 0.01173433, 0.00506078, 0.        , 0.00396242,
        0.00718744],
       [0.02366974, 0.01519446, 0.0088807 , 0.00396242, 0.        ,
        0.00330578],
       [0.02578144, 0.01784393, 0.0119006 , 0.00718744, 0.00330578,
        0.        ]])